In [62]:
!pip install lxml

     |████████████████████████████████| 5.4 MB 4.4 MB/s eta 0:00:01


In [ ]:
# Version 67
# From https://ungoogled-software.github.io/ungoogled-chromium-binaries/releases/linux_portable/64bit/
!wget -q https://github.com/Eloston/ungoogled-chromium-binaries/releases/download/67.0.3396.87-2/ungoogled-chromium_67.0.3396.87-2_linux.tar.xz
!tar -xf ungoogled-chromium_67.0.3396.87-2_linux.tar.xz

# From https://chromedriver.storage.googleapis.com/index.html
!wget -q https://chromedriver.storage.googleapis.com/2.41/chromedriver_linux64.zip
!unzip -o chromedriver_linux64.zip -d ./ungoogled-chromium_67.0.3396.87-1_linux

In [121]:
import requests

def transcript(id='c-7YqBrtuNQ'):
    resp = requests.get(f'https://hierogly.ph/api/transcribe?v={id}',headers={'User-Agent':'Mozilla/5.0'})
    return resp.json()['text']

transcript('GZ7xHi8AIjM')

"A storm is coming. Oh kid: what you gon na do now: [ Music, ] yeah! You got a monster: [ Music, ], [ Applause, ], [, Music, ], the ashes Susan called this great and stop now. Well, I'm sorry [ Music ]! What you do! It's all! [, Music, ] [, Music, ], [ Applause, ], [, Music, ], [, Music, ], [, Music, ], [, Music ]. We are [ Music, ], [, Music, ], [, Music, ], [, Music, ], [, Music, ], [, Music, ], [, Music, ], so sorry, [, Music, ], [, Music, ], [, Music, ], [, Music, ], Oh [, Music, ], [, Music, ], [ Applause, ], [, Music, ] [ Music, ], [, Music, ], [, Music, ], [, Music, ], honey, [, Music ], just oh, this think, [, Music, ], [, Music ], give me out Brogan's and start fights, Angela's percodans, [, Music, ] yeah. It'S fine to me justifies Oh [, Music, ], [, Music, ], [, Music, ], [, Music, ], [ Applause, ], [, Music, ], [ Applause, ], [, Music, ], [ Applause, ], [, Music, ], [, Music, ], [, Music, ], [ Applause, ], [, Music, ], [ Applause, ] [, Music, ], [ Applause, ], [, Music, ], [

In [40]:
from selenium import webdriver
import os
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--incognito')
chrome_options.add_argument('log-level=3')
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--no-sandbox")
chrome_driver = os.getcwd() +"/ungoogled-chromium_67.0.3396.87-1_linux/chromedriver"
browser = webdriver.Chrome(options=chrome_options, executable_path=chrome_driver)

In [156]:
class youtube(object):
    driver = None

    def __init__(self,driver):
        self.driver = driver
        
    def getrecom(self, id):
        self.driver.get(f'https://www.youtube.com/watch?v={id}')
        return self.driver.page_source
    
    def getscreenshot(self):
        self.driver.save_screenshot('test.png')
        return None

In [157]:
yt = youtube(browser)

In [63]:
from lxml import etree

In [158]:
html = etree.HTML(yt.getrecom('cYtSEj1v0ew'))

In [160]:
yt.getscreenshot()

In [159]:
## Get the IDs from all the anchors
[str.split(x.attrib['href'],'=')[1] for x in html.findall(".//a[@id='thumbnail']") if ('href' in x.attrib.keys())]

[]

In [155]:
[x.attrib['href'] for x in html.findall(".//a") if ('href' in x.attrib.keys())]

['https://www.youtube.com',
 '',
 '/',
 '/',
 'https://www.youtube.com/about/',
 'https://www.youtube.com/about/press/',
 'https://www.youtube.com/about/copyright/',
 '/t/contact_us/',
 'https://www.youtube.com/creators/',
 'https://www.youtube.com/ads/',
 'https://developers.google.com/youtube',
 '/t/terms',
 'https://policies.google.com/privacy?hl=en',
 'https://www.youtube.com/about/policies/',
 'https://www.youtube.com/howyoutubeworks?utm_campaign=ytgen&utm_source=ythp&utm_medium=LeftNav&utm_content=txt&u=https%3A%2F%2Fwww.youtube.com%2Fhowyoutubeworks%3Futm_source%3Dythp%26utm_medium%3DLeftNav%26utm_campaign%3Dytgen',
 '/new']

In [139]:
transcript('rb_adz0tkOw')

"Both of us, we plant a lot of strawberries la on each other's lands. What * screams * Hi, I'm Bolin, I'm Qing, I'm Peps Hi, I'm Leah, and this is Zula ChickChats. Okay, hi guys welcome to today's episode of ChickChats. Why Are you all very nervous? Yes, this is my first time on ChickChats Welcome And today's episode we'll be talking about being in the grey area as much requested from the comments section below. Ah So we hear you okay, we will do this episode for you guys. So my first question to you guys, then, is what happened. Okay, whoever wants can share first la What happened. Bolin, Let's start with Bolin. No Qing should start because she's the new person. Why me? Oh my god? Yes, What happened. I met the person through an app Okay, So we hit off like we talk, then after that move to Telegram. But the thing right we went out for so many times right. The places we go to right were like-, not the kind of place that you and your friends would go to. For example, For example, I need 

In [122]:
transcript('5L31Wbrg7tE')

"On one of the days um, we were supposed to have a date .... He came to my house like downstairs. Then I no time to draw eyebrows no time to draw anything right. Then we [ sat ] at the lamp post. There [, I made ] sure to choose the dimmest lamppost Hi. I ’ m Julian Hi. I ’ m Amanda Hi. I ’ m John Hi. I ’ m Leah and this is Zula ChickChats Hi guys welcome to today. ’ s, episode of Zula ChickChats And today we ’ re gon na be talking about- the honeymoon phase in relationships. We have 3 people over here to share uh- their honeymoon phases in their relationships. So…. The first Can you: why Can you explain what the honeymoon phase is I ...? What do you mean You don, ’ t know I ’ m married. I forgot about it. It ’ s been a long time ago. Okay, so the honeymoon phase, I guess, would be the period before your relationship starts getting serious, All the butterflies, all the romantic gestures, Oh ..., For the first question: maybe we ’ ll, let the audience know for you guys. How long have yo

In [140]:
transcript('cYtSEj1v0ew')

"This is John and Jill. They are about to get married and start a family, but first they have to plan for their future and it's not buying the perfect home. Both John and Jill are working polygraph and the earn a combined monthly salary of $ 4,000. Ideally, they would like to live near their parents in the ground, so many options is easy to get confused now. Do you know what options you have, how welcome number option? 1, a 3-bedroom easy in some balance, a 3-bedroom easy and the balance will cost you about. Seven hundred ninety thousand dollars there's a monthly household income of four thousand dollars. You will be able to take a bank loan of up to two hundred and twenty thousand dollars. Next, let's start by it, you can receive a CPF housing grant releases of up to $ 30,000. This review is four hundred ninety thousand dollars, which has to be paid up front and and resupply suspend for thirty. Nine thousand five hundred dollars has to be paid cash, while the rest can be covered by CP

In [145]:
transcript('mPHGGxZwICI')

"My name is I've been working for about two years. My name is Rob. I'Ve been working for about two years, six years, plus I think five years plus I'm very sure, let's see God my location off just nice kata-kata and they realize okay might as well. We just did I just graduated. Yes, I think two years ago yeah - and I guess it just seemed like a natural information from that February. For the column, one poet a key numbers, a thousand by hello, I'm Hansel - I just graduated from in yet last year, and now I'm doing an internship of jewelry come and your student, your tree, we have been together what six? Yes, I think with us always day. Video, what does the you need to be married before you can apply for the blueblack all controller? I mean you're gon na collect the key. Oh boss, you don't need to be married, like yeah, you can apply first, then you get true or false. It is possible to play pain, zero cash yeah. You can actually go share without paying any cash. It wasn't actually, when w